# Convert Promises to Async/Await in the Twilio Node.js Client Library using Codegen

This notebook will show you how to convert a repeated use of a promise chain to async await 
in the official twilio js client library twilio/twilio-node repository.

1. Finds all methods containing operationPromise.then chains
2. Converts the promise chain to use async await
3. Gets rid of the callback handler by adding try catch directly in the function body

In [ ]:
from codegen import Codebase
from codegen.sdk.enums import ProgrammingLanguage
from codegen.sdk.core.statements.statement import StatementType

In [ ]:
print("Initializing codebase...")
codebase = Codebase("twilio/twilio-node", programming_language=ProgrammingLanguage.TYPESCRIPT)
print("Twilio repository initialized!")

### 1. Find all Promise Chains in class methods with the base call being `operationPromise.then`

In [ ]:
# loop through all files -> classes -> methods to find promise the operationPromise chains
operation_promise_chains = []
unique_files = set()

i = 0

for _class in codebase.classes:
    for method in _class.methods:
        if method.name in ["each", "setPromiseCallback"]:
            print("skipping method", method.name, "...")
            continue

        # Only process methods containing operationPromise
        if not method.find("operationPromise"):
            continue

        # Find the first promise chain with then blocks
        for promise_chain in method.promise_chains:
            promise_statement = promise_chain.parent_statement
            operation_promise_chains.append({"function_name": method.name, "promise_chain": promise_chain, "promise_statement": promise_statement})
            unique_files.add(method.file.filepath)
            i += 1
            if i < 10:
                print(f"Found operation promise in the {method.name} method in {method.file.filepath} file.")

In [ ]:
print("Number of Operation Promise Chains found:", len(operation_promise_chains))
print("Number of files affected:", len(unique_files))

### 2. Convert *all* 592 Promise Chains to Async/Await  

*Additionally...* Add a Try/Catch to Eliminate the Callback Handler

In [ ]:
i = 0
assignment_variable_name = "operation"

for promise_chain_dict in operation_promise_chains:
    promise_chain = promise_chain_dict["promise_chain"]
    promise_statement = promise_chain_dict["promise_statement"]
    function_name = promise_chain_dict["function_name"]
    # ---------- CONVERT PROMISE CHAIN TO ASYNC AWAIT ----------
    async_await_code = promise_chain.convert_to_async_await(assignment_variable_name=assignment_variable_name, inplace_edit=False)

    if i < 10:
        print(f"converting {function_name} promise chain to async/await.")

    i += 1
    # ---------- ADD TRY CATCH BLOCK INSTEAD OF CALLBACK HANDLER ----------
    new_code = f"""\
        try {{
            {async_await_code}

            if (callback) {{
                callback(null, {assignment_variable_name});
            }}

            return {assignment_variable_name};
        }} catch(err: any) {{
            if (callback) {{
                callback(err);
            }}
            throw err;
        }}"""

    promise_statement.edit(new_code)

    # ---------- CLEAN UP CALLBACK HANDLER ASSIGNMENT AND SUBSEQUENT RETURN STATEMENT ----------
    statements = promise_statement.parent.get_statements()
    return_stmt = next((stmt for stmt in statements if stmt.statement_type == StatementType.RETURN_STATEMENT), None)
    assign_stmt = next((stmt for stmt in reversed(statements) if stmt.statement_type == StatementType.ASSIGNMENT), None)

    if return_stmt:
        return_stmt.remove()
    if assign_stmt:
        assign_stmt.remove()

codebase.commit()

After making the changes, ensure to run *`npx prettier --write .`* to format line indentation and linting errors.

In [ ]:
# reset the formatting
codebase.reset()

In [ ]:
codebase = Codebase("/Users/tawsifkamal/Documents/codegen-repos/twilio-node", programming_language=ProgrammingLanguage.TYPESCRIPT)